In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
import re

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [26]:
# importo dal df con i risultati Google, in modo da avere già l'elenco delle città
booking_results = pd.read_csv('google_results.csv')
booking_results['risultati'] = np.nan

booking_results


,citta,risultati
0,Austin,NaN
1,Wrocław,NaN
2,Innsbruck,NaN
3,Villefranche-sur-Saône,NaN
4,Zakopane,NaN
...,...,...
2398,Veneto,NaN
2399,Bastia,NaN
2400,Laramie,NaN
2401,Longyearbyen,NaN


In [4]:
# scraping dei risultati della ricerca su Booking, per ogni città

def start_soup(link):
    response = requests.get(link, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


def scraping(df):
    i=1
    for x in df['citta']:
        print(x,i)
        i+=1
        base_link = f"https://www.booking.com/searchresults.en-gb.html?ss={x}"

        try:
            results = start_soup(base_link).find("h1").text
            reg = re.search('(.*): (.*) properties found', results).group(2)
            print(reg)
            # inserisco il numero nel dataframe, in corrispondenza della città
            df['risultati'][df['citta'] == x] = reg
        except:
            df['risultati'][df['citta'] == x] = np.nan
    return df
            
# senza l'header mi restituisce la pagina di accettazione cookie, quindi non riesco a estrarre il numero di risultati
headers = {
    'User-agent':  'Mozilla/5.0 (X11; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0' }

booking_results = scraping(booking_results)

In [29]:
#salvo i risultati, anche se non sono quelli definitivi
booking_results.to_csv('booking_results_not_final.csv', index=False)
# vedo quali sono le città per le quali non sono stati trovati risultati
print(list(booking_results['citta'][booking_results['risultati'].isna()]))

['New York', 'Hyderabad', 'Moscow', 'Novosibirsk', 'Aegean', 'Creully sur Seulles', 'Maui County', 'Gevaş', 'Saint Petersburg', 'Waikoloa Village', 'Jinan City', 'Yinchuan City', 'Gemeinde Sölden', 'Xiamen City', 'Tehran', 'Maoming City', 'Shenyang City', 'Yantai City', 'Bolzano - Bozen', 'Reno Lake', 'Batna', 'Yiyang', 'Haikou City', 'Kharagpur', 'Medina County', 'Wollongong City Council', 'Havana', 'Columbia County', 'Asan-si', 'Hainan Province', 'Kolonnawa', 'Nagano Prefecture', 'Castelló / Castellón', 'Yangzhou City', 'City of Bradford', 'Bhubaneswar (M.Corp.)', 'Bellevue', 'Hongzhou', 'Changchun City', 'Provincia di Trento', 'Regina', 'Larissa', 'Sanya City', 'Wuxi City', 'Poultney', 'Općina Konavle', 'Taoyuan City', 'Constance', 'Yaroslavl', 'Листвянское городское поселение', 'Urbana', 'Kazan', 'Piran / Pirano', 'Whistler Resort Municipality', 'Yekaterinburg', 'City of Saratoga Springs', 'Shiraz', 'Guangdong Province', 'Alabama', 'Albemarle County', 'Mashhad', 'Saasveld', 'Isfaha

In [5]:
"""Osservazioni sulle città senza risultati:
    1) alcune di queste sono russe, e Booking al momento non permette la prenotazione di alloggi
       in Russia.
    2) molte hanno il suffisso "city", che non appartiene al nome vero e proprio della città.
       Lo elimino e rieseguo la ricerca. La rieseguo
       soltanto sulle città che non avevano restituito un valore (è inutie rieseguirlo sulle
       città per le quali ho già dei risultati).
"""

def cut(citta):
    if citta.endswith('City'):
        return re.sub(r"City$", '', citta)
    return citta

booking_results['citta'] = booking_results['citta'].map(cut)

booking_results_no_city_suffix = scraping(booking_results[booking_results['risultati'].isna()])
booking_results_no_city_suffix

New York 1
581
Hyderabad 2


/home/marco/.local/lib/python3.8/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


364
Moscow 3
Novosibirsk 4
Aegean 5
Creully sur Seulles 6
Maui County 7
Gevaş 8
Saint Petersburg 9
Waikoloa Village 10
Jinan  11
98
Yinchuan  12
32
Gemeinde Sölden 13
Xiamen  14
94
Tehran 15
Maoming  16
10
Shenyang  17
66
Yantai  18
41
Bolzano - Bozen 19
Reno Lake 20
Batna 21
Yiyang 22
Haikou  23
68
Kharagpur 24
Medina County 25
Wollongong City Council 26
Havana 27
Columbia County 28
Asan-si 29
Hainan Province 30
Kolonnawa 31
Nagano Prefecture 32
Castelló / Castellón 33
Yangzhou  34
35
City of Bradford 35
Bhubaneswar (M.Corp.) 36
Bellevue 37
Hongzhou 38
Changchun  39
43
Provincia di Trento 40
Regina 41
Larissa 42
Sanya  43
148
Wuxi  44
85
Poultney 45
Općina Konavle 46
Taoyuan  47
35
Constance 48
Yaroslavl 49
Листвянское городское поселение 50
Urbana 51
Kazan 52
Piran / Pirano 53
Whistler Resort Municipality 54
Yekaterinburg 55
City of Saratoga Springs 56
Shiraz 57
Guangdong Province 58
Alabama 59
Albemarle County 60
Mashhad 61
Saasveld 62
Isfahan 63
Togo 64
District of Trnava 65
Town o

,citta,risultati
24,New York,581
34,Hyderabad,364
71,Moscow,NaN
84,Novosibirsk,NaN
97,Aegean,NaN
...,...,...
2378,Akron,NaN
2379,Yalta,NaN
2380,Vladivostok,NaN
2389,Villa Clara,NaN


In [7]:
# vedo se ho ottenuto nuovi valori: ce ne sono 25 (234-209)
print(booking_results_no_city_suffix[booking_results_no_city_suffix['risultati'].isna()])

# aggiungo questi nuovi risultati al primo dataset
final_df = pd.merge(booking_results, booking_results_no_city_suffix, on='citta', how='left')
final_df['risultati_x']= np.where(final_df['risultati_y'].isna(), final_df['risultati_x'], final_df['risultati_y'])

final_df = final_df.drop('risultati_y', axis=1).rename(columns='')

final_df


,citta,risultati_x
0,Austin,639
1,Wrocław,970
2,Innsbruck,288
3,Villefranche-sur-Saône,16
4,Zakopane,"2,227"
...,...,...
2398,Veneto,"14,714"
2399,Bastia,177
2400,Laramie,33
2401,Longyearbyen,12


In [100]:
final_df.to_csv('booking_results.csv', index=False)